In [ ]:
!pip install praw
import praw
import config

reddit = praw.Reddit(
    client_id=config.CLIENT_ID,
    client_secret=config.CLIENT_SECRET,
    user_agent=config.USER_AGENT,
    username=config.USERNAME,
    password=config.PASSWORD
)

  Obtaining dependency information for praw from https://files.pythonhosted.org/packages/73/ca/60ec131c3b43bff58261167045778b2509b83922ce8f935ac89d871bd3ea/praw-7.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for prawcore<3,>=2.4 from https://files.pythonhosted.org/packages/96/5c/8af904314e42d5401afcfaff69940dc448e974f80f7aa39b241a4fbf0cf1/prawcore-2.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for update_checker>=0.18 from https://files.pythonhosted.org/packages/0c/ba/8dd7fa5f0b1c6a8ac62f8f57f7e794160c1f86f31c6d0fb00f582372a3e4/update_checker-0.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client>=0.54.0 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16141199bf9acb7537a3/websocket_client-1.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for requests<3.0,>=2.6.0 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d

In [3]:
subreddit = reddit.subreddit("philosophy")
for post in subreddit.hot(limit=10):
    print(post.title)

Welcome to /r/philosophy! Check out our rules and guidelines here. [June 1 2024 Update]
/r/philosophy Open Discussion Thread | May 19, 2025
Article: "Why Marxists Need Foucault"; Foucault helps Marxist understand how ideology works today - linking identity struggles with class domination.
Alasdair MacIntyre, Author of After Virtue, has died at 96.
Talking to Machines, Learning to Be Human: AI as a Moral Feedback Loop
Chinese philosophy challenges the foundations of reality. | The fundamental structure of reality is not fixed or singular, but a kaleidoscope of partial truths, each offering a legitimate, yet incomplete, perspective shaped by human attachment and context.
Derrida, Badiou, Baudrillard: Three Thinkers of the LLM
Antinatalist philosopher: The Palm Springs bomber proves my point
Ruth Chang suggests if we’re stuck in a hard life choice, we shouldn’t fool ourselves into thinking there must be a ‘best’ path; often there are simply different paths, which will change us in differe

In [ ]:
!pip install pandas
import pandas as pd
import time

def fetch_posts(subreddit_name, limit=500):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []

    for post in subreddit.top(limit=limit, time_filter="month"):
        if not post.stickied and not post.locked:
            posts.append({
                "subreddit": subreddit_name,
                "id": post.id,
                "title": post.title,
                "text": post.selftext,
                "author": str(post.author) if post.author else "[deleted]",
                "score": post.score,
                "created_utc": post.created_utc,
                "num_comments": post.num_comments,
                "url": post.url
            })
        time.sleep(0.2)

    return pd.DataFrame(posts)

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/52/11/9eac327a38834f162b8250aab32a6781339c69afe7574368fffe46387edf/pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.2 MB/s eta 0:00:00
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/packages/4a/9f/0121e375000b5e50ffdd8b25bf78d8e1a5aa4cca3f185d41265198c7b834/numpy-2.2.6-cp311-cp311-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.5 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/5c/23/c7abc0ca0a1526a0774eca151daeb8de62ec457e77262b66b359c3c7679e/tzdata-2025.2-py2.py3-none-any.whl

In [6]:
subs = ["philosophy", "askphilosophy", "sociology", "stoicism", "TrueAskReddit"]
all_posts = pd.DataFrame()

for sub in subs:
    print(f"Fetching from r/{sub}...")
    df = fetch_posts(sub, limit=500)
    all_posts = pd.concat([all_posts, df], ignore_index=True)

# Save to CSV
all_posts.to_csv("reddit_philosophy_data.csv", index=False)
print("Saved all posts to reddit_philosophy_data.csv")

Fetching from r/philosophy...
Fetching from r/askphilosophy...
Fetching from r/sociology...
Fetching from r/stoicism...
Fetching from r/TrueAskReddit...


OSError: [Errno 30] Read-only file system: 'reddit_philosophy_data.csv'

In [7]:
df = pd.read_csv("reddit_philosophy_data.csv")
print(df.shape)
df.sample(5)

FileNotFoundError: [Errno 2] No such file or directory: 'reddit_philosophy_data.csv'

In [8]:
import re

def clean_text(text):
    if pd.isnull(text):
        return ""
    text = re.sub(r"http\S+", "", text)        
    text = re.sub(r"\[.*?\]\(.*?\)", "", text)  
    text = re.sub(r"[^\w\s]", "", text)     
    text = re.sub(r"\s+", " ", text)                
    return text.strip().lower()

df["clean_title"] = df["title"].apply(clean_text)
df["clean_text"] = df["text"].apply(clean_text)
df["full_text"] = df["clean_title"] + " " + df["clean_text"]

In [9]:
df[["subreddit", "clean_title", "clean_text"]].sample(5)

,subreddit,clean_title,clean_text
5,TrueAskReddit,is karma real or just a comforting illusion we...,ive been thinking a lot about the concept of k...
15,TrueAskReddit,why do some people think surrogacy is wrong,i dont have kids so im just genuinely curious ...
9,TrueAskReddit,what happens when ai is used in war,ai in war isnt just science fiction anymore it...
18,TrueAskReddit,why are we more likely to help those we can se...,the reason we or other animals have perception...
16,TrueAskReddit,why do you think so many people are burned out...,


In [10]:
!python3 -m spacy download en_core_web_sm

/Users/tylersear/.pyenv/versions/3.11.5/bin/python3: No module named spacy


In [11]:
!pip install scikit-learn
!pip install spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_relevant_words(text):
    if not isinstance(text, str):
        return ""
    doc = nlp(text)
    return " ".join(
        token.lemma_.lower()
        for token in doc
        if token.pos_ in {"NOUN", "PROPN", "ADJ", "VERB"}
        and not token.is_stop
        and token.is_alpha
    )

df["content_words"] = df["full_text"].apply(extract_relevant_words)
vectorizer = TfidfVectorizer(
    max_features=50,
    min_df=2, 
    max_df=0.8 
)

subreddit_docs = df.groupby("subreddit")["content_words"].apply(lambda x: " ".join(x)).to_dict()

tfidf_matrix = vectorizer.fit_transform(subreddit_docs.values())
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=subreddit_docs.keys(), columns=vectorizer.get_feature_names_out())

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.heatmap(tfidf_df.T, cmap="magma", annot=True)
plt.title("Refined TF-IDF Keyword Strength by Subreddit")
plt.xlabel("Subreddit")
plt.ylabel("Keyword")
plt.tight_layout()
plt.show()

noise_terms = {"thank", "start", "bring", "happen", "use", "thing", "stuff", "someone", "everyone", "anyone"}

tfidf_df_cleaned = tfidf_df.drop(columns=[col for col in tfidf_df.columns if col in noise_terms])

plt.figure(figsize=(12, 8))
sns.heatmap(tfidf_df_cleaned.T, cmap="magma", annot=True)
plt.title("Filtered TF-IDF: Discourse-Significant Terms Only")
plt.xlabel("Subreddit")
plt.ylabel("Keyword")
plt.tight_layout()
plt.savefig("tfidf_discourse_heatmap.png", dpi=300, bbox_inches="tight")
plt.show()

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/25/92/ee1d7a00bb6b8c55755d4984fd82608603a3cc59959245068ce32e7fb808/scikit_learn-1.6.1-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/4b/fa/a7e5b95afd80d24313307f03624acc65801846fa75599034f8ceb9e2cbf6/scipy-1.15.3-cp311-cp311-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.2 MB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/7d/4f/1195bbac8e0c2acc5f740661631d8d750dc38d4a32b23ee5df3cde6f4e0d/joblib-1.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/32/d5/f9a850d79b0851d1d4ef6456097579a9005b31fea68726a4ae5f2d82ddd9/threadpoolctl-3.6.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [12]:
!pip install spacy
from collections import Counter
import itertools
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_concepts(text):
    if not isinstance(text, str):
        return []
    doc = nlp(text)
    return [token.lemma_.lower() for token in doc 
            if token.pos_ in {"NOUN", "PROPN"}
            and not token.is_stop
            and token.is_alpha]

df["concepts"] = df["full_text"].apply(extract_concepts)


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
from collections import defaultdict

co_occurrence = defaultdict(int)

for concepts in df["concepts"]:
    unique_terms = list(set(concepts))
    for pair in itertools.combinations(sorted(unique_terms), 2):
        co_occurrence[pair] += 1

KeyError: 'concepts'

In [14]:
!pip install networkx
import networkx as nx

G = nx.Graph()

for (term1, term2), weight in co_occurrence.items():
    if weight >= 3:
        G.add_edge(term1, term2, weight=weight)

  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/b9/54/dd730b32ea14ea797530a4479b2ed46a6fb250f682a9cfb997e968bf0261/networkx-3.4.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.9 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [15]:
degree_dict = dict(nx.degree(G))
print("Max degree:", max(degree_dict.values()))
print("Min degree:", min(degree_dict.values()))

top_nodes = sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)[:50]
nodes_to_keep = [node for node, _ in top_nodes]

G_filtered = G.subgraph(nodes_to_keep).copy()

print("Filtered nodes:", len(G_filtered.nodes))

ValueError: max() arg is an empty sequence

In [16]:
min_edge_weight = 35

G_strong = nx.Graph()

for u, v, data in G_filtered.edges(data=True):
    if data["weight"] >= min_edge_weight:
        G_strong.add_edge(u, v, **data)

print("Nodes after filtering:", len(G_strong.nodes))
print("Edges after filtering:", len(G_strong.edges))

NameError: name 'G_filtered' is not defined

In [17]:
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 14))
pos = nx.spring_layout(G_strong, k=0.9, iterations=150)

nx.draw_networkx_nodes(G_strong, pos, node_size=500, node_color="orange", alpha=0.85)
nx.draw_networkx_edges(G_strong, pos, width=1.5, alpha=0.4)
nx.draw_networkx_labels(G_strong, pos, font_size=11)

plt.title("Core Concept Co-Occurrence Network", fontsize=16)
plt.axis("off")
plt.tight_layout()
plt.savefig("concept_network.png", dpi=300, bbox_inches="tight")
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [18]:
!pip install nltk

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/c5/1b/f0e4d13e6adf866ce9b069e191f303a30ab1277e037037a365c3aad5cc9c/regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 23.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tylersear/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
df["sentiment"] = df["full_text"].apply(lambda x: sia.polarity_scores(x)["compound"])

In [20]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.boxplot(x="subreddit", y="sentiment", data=df)
plt.title("Sentiment Distribution by Subreddit")
plt.xlabel("Subreddit")
plt.ylabel("Compound Sentiment Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("sentiment_dist.png", dpi=300, bbox_inches="tight")
plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [21]:
import zipfile

with zipfile.ZipFile("NRC-Emotion-Lexicon.zip", "r") as zip_ref:
    zip_ref.extractall("nrc_emotion_lexicon")

FileNotFoundError: [Errno 2] No such file or directory: 'NRC-Emotion-Lexicon.zip'

In [22]:
import os

for root, dirs, files in os.walk("nrc_emotion_lexicon"):
    for file in files:
        if "Wordlevel-v0.92.txt" in file:
            full_path = os.path.join(root, file)
            print("✅ FOUND:", full_path)

In [23]:
emotion_lexicon = {}

with open("nrc_emotion_lexicon/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt", "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) == 3:
            word, emotion, score = parts
            if int(score) == 1:
                emotion_lexicon.setdefault(word, set()).add(emotion)

print(list(emotion_lexicon.items())[:10])

FileNotFoundError: [Errno 2] No such file or directory: 'nrc_emotion_lexicon/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

In [24]:
from collections import Counter

def get_emotions(text):
    if not isinstance(text, str):
        return Counter()
    words = text.lower().split()
    emotions = []
    for word in words:
        if word in emotion_lexicon:
            emotions.extend(emotion_lexicon[word])
    return Counter(emotions)

df["emotion_counts"] = df["full_text"].apply(get_emotions)

In [25]:
emotion_df = df["emotion_counts"].apply(pd.Series).fillna(0)

emotion_df["subreddit"] = df["subreddit"]

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns

emotion_means = emotion_df.groupby("subreddit").mean()

plt.figure(figsize=(12, 6))
sns.heatmap(emotion_means.T, cmap="coolwarm", annot=True)
plt.title("Average Emotion Score per Subreddit")
plt.xlabel("Subreddit")
plt.ylabel("Emotion")
plt.tight_layout()
plt.savefig("indepth_sentiment_dist.png", dpi=300, bbox_inches="tight")
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [27]:
known_names = {
    "nietzsche", "kant", "plato", "socrates", "aristotle", "foucault", 
    "camus", "locke", "hume", "spinoza", "marx", "zizek", "aurelius", 
    "descartes", "heidegger", "sartre", "chomsky", "rawls", "mill", "hobbes"
}

In [28]:
from collections import Counter

def match_known_names(text):
    if not isinstance(text, str): return []
    words = set(text.lower().split())
    return [name for name in known_names if name in words]

df["figures_cited"] = df["full_text"].apply(match_known_names)
name_counts = Counter([name for sublist in df["figures_cited"] for name in sublist])
top_cited = name_counts.most_common(20)

for name, count in top_cited:
    print(f"{name.title()}: {count}")

In [29]:
figure_emotion_profiles = {
    "aurelius":   {"trust": 0.8, "joy": 0.4, "sadness": 0.1, "fear": 0.1, "positive": 0.6, "negative": 0.1},
    "kant":       {"trust": 0.7, "joy": 0.1, "fear": 0.2, "anticipation": 0.4, "positive": 0.3},
    "plato":      {"trust": 0.5, "anticipation": 0.3, "joy": 0.2},
    "nietzsche":  {"anger": 0.4, "fear": 0.5, "sadness": 0.3, "negative": 0.6},
    "aristotle":  {"trust": 0.6, "joy": 0.3, "positive": 0.5},
    "socrates":   {"trust": 0.7, "joy": 0.2, "surprise": 0.2},
    "camus":      {"sadness": 0.5, "disgust": 0.3, "negative": 0.6},
    "descartes":  {"trust": 0.6, "anticipation": 0.4},
    "marx":       {"anger": 0.4, "disgust": 0.3, "sadness": 0.3, "negative": 0.5},
    "spinoza":    {"joy": 0.4, "trust": 0.5, "positive": 0.5},
    "hume":       {"trust": 0.6, "joy": 0.3, "positive": 0.4},
    "sartre":     {"sadness": 0.4, "fear": 0.3, "negative": 0.5},
    "heidegger":  {"fear": 0.5, "disgust": 0.3, "negative": 0.4},
    "foucault":   {"fear": 0.4, "disgust": 0.3, "sadness": 0.2, "negative": 0.5},
    "hobbes":     {"fear": 0.5, "anger": 0.4, "negative": 0.5},
    "rawls":      {"trust": 0.6, "anticipation": 0.3, "positive": 0.4},
    "locke":      {"trust": 0.6, "joy": 0.2, "positive": 0.3},
    "chomsky":    {"anger": 0.3, "trust": 0.5, "fear": 0.2},
}

In [30]:
emotion_df = df["emotion_counts"].apply(pd.Series)
df = pd.concat([df, emotion_df], axis=1)

In [31]:
emotion_cols = ["positive", "negative", "anger", "fear", "joy", "sadness", "trust", "anticipation", "disgust", "surprise"]

df_emotions = df[emotion_cols].copy()
df_emotions["figures_cited"] = df["figures_cited"]
df_emotions["subreddit"] = df["subreddit"]

df_figure_mention = df_emotions.explode("figures_cited").dropna(subset=["figures_cited"])

KeyError: "None of [Index(['positive', 'negative', 'anger', 'fear', 'joy', 'sadness', 'trust',\n       'anticipation', 'disgust', 'surprise'],\n      dtype='object')] are in the [columns]"

In [32]:
from scipy.spatial.distance import euclidean
import numpy as np

def compare_to_figure_profile(row):
    figure = row.name[1]
    if figure in figure_emotion_profiles:
        figure_profile = figure_emotion_profiles[figure]
        
        shared_emotions = set(row.index) & set(figure_profile.keys())
        if shared_emotions:
            subreddit_vector = [row[emotion] for emotion in shared_emotions]
            figure_vector = [figure_profile[emotion] for emotion in shared_emotions]
            
            if any(pd.isna(v) for v in subreddit_vector):
                return None
            
            return euclidean(subreddit_vector, figure_vector)
    
    return None

sub_cite_emotions = df_figure_mention.groupby(["subreddit", "figures_cited"])[emotion_cols].mean()

sub_cite_emotions["emotion_distance"] = sub_cite_emotions.apply(compare_to_figure_profile, axis=1)

NameError: name 'df_figure_mention' is not defined

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns

plot_df = sub_cite_emotions.reset_index()

plot_df = plot_df.dropna(subset=["emotion_distance"])

plot_df = plot_df.sort_values("emotion_distance", ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=plot_df, x="figures_cited", y="emotion_distance", hue="subreddit")

plt.title("Emotional Distance Between Subreddits and Cited Figures")
plt.xlabel("Cited Figure")
plt.ylabel("Euclidean Distance in Emotion Profile")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("Cited-figures-subreddit.png", dpi=300, bbox_inches="tight")
plt.show()

distance_df = plot_df

ModuleNotFoundError: No module named 'matplotlib'

In [34]:
print(distance_df["subreddit"].unique())
print(distance_df["subreddit"].value_counts())

NameError: name 'distance_df' is not defined

In [35]:
import pandas as pd
import numpy as np

In [36]:
emotion_cols = ["positive", "negative", "anger", "fear", "joy", "sadness", "trust", "anticipation", "disgust", "surprise"]

# 1. Average emotion intensity per subreddit
emotion_means = df.groupby("subreddit")[emotion_cols].mean()

# 2. TF-IDF uniqueness: how many high-weight keywords are used
tfidf_uniqueness = tfidf_df.apply(lambda row: (row > 0.1).sum(), axis=1)

# 3. Citation frequency per subreddit
citation_counts = df["figures_cited"].explode().groupby(df["subreddit"]).count()

# 4. Mean emotional distance to cited figures
mean_emotion_distance = plot_df.groupby("subreddit")["emotion_distance"].mean()

KeyError: "Columns not found: 'fear', 'positive', 'disgust', 'negative', 'anger', 'anticipation', 'surprise', 'sadness', 'trust', 'joy'"

In [37]:
subreddit_summary = pd.DataFrame({
    "tfidf_uniqueness": tfidf_uniqueness,
    "citation_count": citation_counts,
    "mean_emotion_distance": mean_emotion_distance
})

subreddit_summary = subreddit_summary.join(emotion_means)

subreddit_summary = subreddit_summary.fillna(0)

NameError: name 'tfidf_uniqueness' is not defined

In [38]:
print(subreddit_summary.columns)
print(subreddit_summary.loc["philosophy"])

NameError: name 'subreddit_summary' is not defined

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
normalized_summary = pd.DataFrame(
    scaler.fit_transform(subreddit_summary),
    columns=subreddit_summary.columns,
    index=subreddit_summary.index
)

NameError: name 'subreddit_summary' is not defined

In [40]:
normalized_summary.head()

NameError: name 'normalized_summary' is not defined

In [41]:
import matplotlib.pyplot as plt
import numpy as np

def plot_radar(data, subreddit, save=False):
    import matplotlib.pyplot as plt
    import numpy as np

    values = data.loc[subreddit].values
    labels = data.columns.tolist()

    # Repeat first value to close the loop
    values = np.concatenate((values, [values[0]]))
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.plot(angles, values, linewidth=2)
    ax.fill(angles, values, alpha=0.25)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=10)
    ax.set_yticklabels([])
    ax.set_title(f"Discourse Profile: {subreddit}", fontsize=14)

    plt.tight_layout()

    if save:
        filename = f"radar_{subreddit}.png"
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        print(f"✅ Saved: {filename}")

    plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [42]:
plot_radar(normalized_summary, "askphilosophy", save=True)

NameError: name 'plot_radar' is not defined

In [43]:
for sub in normalized_summary.index:
    plot_radar(normalized_summary, sub, save=True)

NameError: name 'normalized_summary' is not defined

In [44]:
top_keywords = {}

for sub in tfidf_df.index:
    top_terms = tfidf_df.loc[sub]
    top_terms = top_terms[top_terms > 0]  # Only meaningful terms
    top_keywords[sub] = top_terms.sort_values(ascending=False).head(5).index.tolist()

NameError: name 'tfidf_df' is not defined

In [45]:
def plot_radar(data, subreddit, top_keywords_dict=top_keywords):
    labels = data.columns.tolist()
    values = data.loc[subreddit].values.tolist()
    values += values[:1]  # loop around

    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

    ax.plot(angles, values, linewidth=2)
    ax.fill(angles, values, alpha=0.25)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    ax.set_title(f"Discourse Profile: {subreddit}", y=1.1)

    # ✅ Show keywords at the bottom, inside the figure bounds
    keywords = ", ".join(top_keywords_dict.get(subreddit, []))
    fig.text(0.5, 0.02, f"Top terms: {keywords}", ha="center", fontsize=10)

    # 🧼 REMOVE tight_layout — it clips text
    # plt.tight_layout()

    plt.savefig(f"radar_{subreddit}.png", dpi=300, bbox_inches="tight")
    plt.show()

In [46]:
for sub in normalized_summary.index:
    plot_radar(normalized_summary, sub, top_keywords)

NameError: name 'normalized_summary' is not defined

In [47]:
# 1. Emotional intensity = std deviation of emotion scores
emotion_cols = ["positive", "negative", "anger", "fear", "joy", "sadness", "trust", "anticipation", "disgust", "surprise"]
subreddit_summary["emotion_intensity"] = df[emotion_cols].groupby(df["subreddit"]).std().mean(axis=1)

# 2. Emotional temperature (positive - negative)
subreddit_summary["emotional_temp"] = df.groupby("subreddit")["positive"].mean() - df.groupby("subreddit")["negative"].mean()

# 3. Normalize relevant columns for scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaled_cols = ["tfidf_uniqueness", "citation_count", "mean_emotion_distance", "emotion_intensity", "emotional_temp"]
normalized = pd.DataFrame(
    scaler.fit_transform(subreddit_summary[scaled_cols]),
    columns=[f"{col}_scaled" for col in scaled_cols],
    index=subreddit_summary.index
)

# 4. Merge normalized back in
subreddit_summary = pd.concat([subreddit_summary, normalized], axis=1)

KeyError: "None of [Index(['positive', 'negative', 'anger', 'fear', 'joy', 'sadness', 'trust',\n       'anticipation', 'disgust', 'surprise'],\n      dtype='object')] are in the [columns]"

In [48]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Sample DataFrame (replace with your real values)
compass_df = pd.DataFrame({
    "subreddit": ["stoicism", "askphilosophy", "philosophy", "TrueAskReddit", "sociology"],
    "x": [0.1, 0.35, 0.1, 1.0, 0.3],
    "y": [1.0, 0.48, 0.0, 0.8, 0.44],
    "size": [300, 500, 100, 600, 400],
    "color": [0.7, 0.8, 0.2, 1.0, 0.6]
})

plt.figure(figsize=(10, 8))
ax = plt.gca()

# Quadrant background colors
colors = {
    "top_left": "#c7d5e0",
    "top_right": "#faebd7",
    "bottom_left": "#dcdcdc",
    "bottom_right": "#f2f2f2"
}

# Draw quadrant rectangles
ax.axhspan(0.5, 1, xmin=0, xmax=0.5, facecolor=colors["top_left"], alpha=0.6, zorder=0)
ax.axhspan(0.5, 1, xmin=0.5, xmax=1, facecolor=colors["top_right"], alpha=0.6, zorder=0)
ax.axhspan(0, 0.5, xmin=0, xmax=0.5, facecolor=colors["bottom_left"], alpha=0.6, zorder=0)
ax.axhspan(0, 0.5, xmin=0.5, xmax=1, facecolor=colors["bottom_right"], alpha=0.6, zorder=0)

# Plot subreddit bubbles
scatter = ax.scatter(
    compass_df["x"], compass_df["y"],
    s=compass_df["size"],
    c=compass_df["color"],
    cmap="coolwarm",
    alpha=0.9,
    edgecolors='black'
)

# Subreddit name labels
for i, row in compass_df.iterrows():
    ax.text(
        row["x"], row["y"] + 0.05,
        row["subreddit"],
        ha="center", va="bottom",
        fontsize=10, color="black", weight="bold"
    )

# Add quadrant labels with semi-transparent background
def label_quadrant(x, y, text):
    ax.text(
        x, y, text,
        ha="center", va="center",
        fontsize=10, color="black", weight="bold",
        bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", boxstyle="round,pad=0.3")
    )

label_quadrant(0.25, 0.75, "Detached / Reflective")
label_quadrant(0.75, 0.75, "Expressive / Experiential")
label_quadrant(0.25, 0.25, "Detached / Cold")
label_quadrant(0.75, 0.25, "Ideological / Abstract")

# Axis dividers
ax.axvline(0.5, color="gray", linestyle="--", linewidth=1)
ax.axhline(0.5, color="gray", linestyle="--", linewidth=1)
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(-0.05, 1.05)

# Axis labels & title
ax.set_title("🧭 The Discourse Compass: Mapping Reddit Philosophy Subcultures", fontsize=14, weight="bold", pad=20)
ax.set_xlabel("Conceptual Anchoring (Closer to Cited Thinkers →)", fontsize=11)
ax.set_ylabel("Emotional Expressiveness (Muted → Intense)", fontsize=11)

# Add color legend
norm = plt.Normalize(0, 1)
sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label("Emotional Temperature", fontsize=10)

# Final layout
plt.tight_layout()
plt.savefig("discourse_compass_final_refined.png", dpi=300)
plt.show()

ModuleNotFoundError: No module named 'matplotlib'